## Projeto ADA módulo de Extração de dados
## ETL para Powerbi 
https://app.powerbi.com/view?r=eyJrIjoiMmZhNWE0ZDMtZTdmZi00ZGU4LTkzZTQtMGZjOGIyNzMxMzg2IiwidCI6ImUwNzhhODdiLTk5OTQtNGQyZS05OWE2LWYwYzg1ZjU4MjJlMiJ9


In [2]:

import warnings
warnings.filterwarnings("ignore")

import requests

import pandas as pd
import numpy as np

## Funções 
def obter_prduto():
    url = f"https://fakestoreapi.com/products/"
    response = requests.get(url)

    if response.status_code == 200:
        dados = response.json()
        return dados
    else:
        print(f"Erro: {response.status_code}")
        return None

## Adicionar
def obter_carrinhos():
    url = f"https://fakestoreapi.com/carts/"
    response = requests.get(url)

    if response.status_code == 200:
        dados = response.json()
        return dados
    else:
        print(f"Erro: {response.status_code}")
        return None


# Produtos

In [3]:
## Extract
## Obter os dados via api

registros = 20
df_fim = pd.DataFrame()
for i in range(registros):
    # print (i)
    # chamada api
    resultado = obter_prduto()
    df=pd.DataFrame(resultado)
    #Expandir a coluna 'rating' em duas colunas # TRansform
    df[['rate', 'count']] = df['rating'].apply(pd.Series)
    # Remover a coluna original 
    df.drop(columns='rating', inplace=True)
    df_fim = df_fim.append(df)
    del df
# df_fim.head(2)

In [4]:
#transform
#Criando colunas
df_fim['rate'] = df_fim['rate'].sample(frac=1).reset_index(drop=True)
df_fim['count'] = df_fim['rate'].sample(frac=1).reset_index(drop=True)

df_fim['vendidos']   = [np.random.uniform(0, 5000) for _ in range(df_fim.shape[0])]
df_fim['total_faturado'] = df_fim['vendidos']  * df_fim['price']
df_fim['faturado_total(%)_base'] =(df_fim['total_faturado'] / df_fim['total_faturado'].sum())*100

df_fim['rate_peso'] = (df_fim['rate']  * df_fim['count'])
df_fim['ranking_avaliação_base'] = (df_fim['rate']  * df_fim['count']) / df_fim['count'].sum()


df_fim['loja'] = np.random.choice(['Loja 1', 'Loja 2', 'Loja 3'], size = df_fim.shape[0])
# Usando numpy.select para atribuir valores
df_fim['fator_mult_loja'] = np.select([df_fim['loja'] == 'Loja 1',
                                       df_fim['loja'] == 'Loja 2',
                                       df_fim['loja'] == 'Loja 3'], 
                                       [1.3, 0.95, 0.9], default=np.nan)

df_fim['latitudes']  = [np.random.uniform(-90, 90) for _ in range(df_fim.shape[0])]
df_fim['longitudes']   = [np.random.uniform(-180, 180) for _ in range(df_fim.shape[0])]

df_fim['Data'] = pd.to_datetime(np.random.choice(pd.date_range("2023-09-01", '2023-09-30'), size=df_fim.shape[0]))
df_fim['Data'] = pd.to_datetime(df_fim['Data']).dt.strftime('%Y/%m/%d')

df_fim.rename(columns={'id': 'productId'}, inplace=True)
df_fim.head(2)


,productId,title,price,description,category,image,rate,count,vendidos,total_faturado,faturado_total(%)_base,rate_peso,ranking_avaliação_base,loja,fator_mult_loja,latitudes,longitudes,Data
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,2.6,4.1,627.98837,69047.321229,0.044450,10.66,0.007658,Loja 3,0.9,-1.117365,151.824837,2023/09/15
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,2.9,2781.93080,62037.056843,0.039937,11.89,0.008542,Loja 1,1.3,-87.917809,-98.819914,2023/09/30


# Carrinhos

In [5]:
#Extract
### Obter carrinhos
carrinhos = obter_carrinhos()
carrinhos = pd.DataFrame(carrinhos)


In [6]:
#transform
# transformando a base em uma base de produtos no carrinho

# Expandir
carrinhos = carrinhos.explode('products')
#juntar as infos
carrinhos = pd.concat([carrinhos,carrinhos['products'].apply(pd.Series)],axis=1).drop(['products','__v'], axis= 1)

## corrigir data
carrinhos['Data'] = pd.to_datetime(carrinhos['date'])
carrinhos['Data'] = carrinhos['Data'].apply(lambda x: x.replace(year=2023))
carrinhos['Data'] = carrinhos['Data'].apply(lambda x: x.replace(month=9) if x.month == 3 else x)
carrinhos['Data'] = pd.to_datetime(carrinhos['date']).dt.strftime('%Y/%m/%d')


carrinhos.drop(columns = 'date', inplace=True)

# Criar info de loja no df de carrinhos
carrinhos['loja'] = np.random.choice(['Loja 1', 'Loja 2', 'Loja 3'], size = carrinhos.shape[0])

# corrigir nomes
carrinhos = carrinhos.rename(columns={'id': 'id_carrinho'})


In [13]:
#Load  (salvando os dados combinados em parquet)
pronto_load = pd.merge(df_fim,carrinhos, on = ['productId','loja','Data'] , how = 'left')
pronto_load.to_parquet("Dados_projetoETL.parquet", compression='snappy')


In [10]:
# #Gerar arquivo do PBI 
# from powerbiclient import QuickVisualize, get_dataset_config, Report
# from powerbiclient.authentication import DeviceCodeLoginAuthentication

# # Initiate device flow authentication
# auth_device= DeviceCodeLoginAuthentication()

# # Create a Power BI report from your data
# PBI_visualize = QuickVisualize(get_dataset_config(pronto_load.fillna(0)), auth=auth_device)

# # Render the new report
# PBI_visualize

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code B8DV5DNT5 to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


QuickVisualize()